In [ ]:
#贝叶斯优化超参数
import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
!pip install ax-platform
from ax.plot.contour import plot_contour
from ax.plot.trace import optimization_trace_single_method
from ax.service.managed_loop import optimize
from ax.utils.notebook.plotting import render
from ax.utils.tutorials.cnn_utils import train, evaluate
import pandas as pd
#import datainfor
from torch.utils.data import Dataset,DataLoader
import torch
import torch.nn as nn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import pandas as pd
from torch.utils.data import Dataset,DataLoader
from PIL import Image
import torch.optim as optim
from sklearn.metrics import accuracy_score,multilabel_confusion_matrix
from torch.optim import lr_scheduler
#import npmetrics
#import multi_model512_先验_3 as multi_model_128
#from src.loss_functions.losses_调整 import AsymmetricLoss,FocalLoss,AsymmetricLossOptimized,Asy1#引入损失函数
#import index
from torch.backends import cudnn
cudnn.benchmark = False            # if benchmark=True, deterministic will be False
cudnn.deterministic = True
seed=1
torch.manual_seed(seed)            # 为CPU设置随机种子
torch.cuda.manual_seed(seed)       # 为当前GPU设置随机种子
torch.cuda.manual_seed_all(seed)   # 为所有GPU设置随机种子
import random
import numpy as np
random.seed(seed)
np.random.seed(seed)


In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Thu Feb 23 16:10:27 2023

@author: lisha
"""
from __future__ import print_function, division
#import torchsummary
import torch
import torch.nn as nn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import pandas as pd
from torch.utils.data import Dataset,DataLoader
from PIL import Image
import torch.optim as optim
from sklearn.metrics import accuracy_score,multilabel_confusion_matrix
from torch.optim import lr_scheduler
# npmetrics
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
lamda=0.01
num_class=5
# Transformer Parameters
d_model=128# Embedding Size,512-1024,128-512
q_model=256
d_ff=1024# FeedForward dimension
n_layers=4  # number of Encoder of Decoder Layer
n_heads=4  # number of heads in Multi-Head Attention
d_k = d_v =int(d_model/n_heads) # dimension of K(=Q), V
batch_size=50
num_fea=3

class ScaledDotProductAttention(nn.Module):
    def __init__(self):
        super(ScaledDotProductAttention, self).__init__()

    def forward(self, Q, K, V):
        '''
        Q: [batch_size, n_heads, len_q, d_k]
        K: [batch_size, n_heads, len_k, d_k]
        V: [batch_size, n_heads, len_v(=len_k), d_v]
        attn_mask: [batch_size, n_heads, seq_len, seq_len]
    
        '''
        #print('11')
        scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(d_k) 
        #print('scores',scores.size())
        #将K的最后两个维度进行转置
        # scores : [batch_size, n_heads, len_q, len_k]
        attn = nn.Softmax(dim=-1)(scores)
        #print('attn',attn.size())
        #attn就是注意力权重
        context = torch.matmul(attn, V) 
        # [batch_size, n_heads, len_q, d_v]
        #print('context',context.size())
        #print('1')
        return context, attn


class MultiHeadAttention(nn.Module):
    def __init__(self):
        super(MultiHeadAttention, self).__init__()
        self.W_Q = nn.Linear(q_model, d_k * n_heads, bias=False)
        self.W_K = nn.Linear(d_model, d_k * n_heads, bias=False)
        self.W_V = nn.Linear(d_model, d_v * n_heads, bias=False)
        self.fc = nn.Linear(n_heads * d_v, d_model, bias=False)
        self.nor=nn.LayerNorm(d_model)
    def forward(self, input_Q, input_K, input_V):
        '''
        input_Q: [batch_size, len_q, d_model]
        input_K: [batch_size, len_k, d_model]
        input_V: [batch_size, len_v(=len_k), d_model]
        attn_mask: [batch_size, seq_len, seq_len]
        '''
       
        batch_size=input_Q.size(0)
        #input_Q=input_Q.view(batch_size, -1, d_model)
        residual=input_K
        
        #print('residual',residual.size())
        # (B, S, D) -proj-> (B, S, D_new) -split-> (B, S, H, W) -trans-> (B, H, S, W)
        # (B,D) -proj-> (B, D_new) -split-> (B, H, W)
        #改变输入到合适的形式
        # Q = self.W_Q(input_Q).view(batch_size, n_heads, d_k)
        # # # Q: [batch_size, n_heads, len_q, d_k]
        # K = self.W_K(input_K).view(batch_size, n_heads, d_k)  
        # # # K: [batch_size, n_heads, len_k, d_k]
        # V = self.W_V(input_V).view(batch_size, n_heads, d_v)
        # V: [batch_size, n_heads, len_v(=len_k), d_v]
        Q = self.W_Q(input_Q).view(batch_size, -1, n_heads, d_k).transpose(1,2)  # Q: [batch_size, n_heads, len_q, d_k]
        K = self.W_K(input_K).view(batch_size, -1, n_heads, d_k).transpose(1,2)  # K: [batch_size, n_heads, len_k, d_k]
        V = self.W_V(input_V).view(batch_size, -1, n_heads, d_v).transpose(1,2)  # V: [batch_size, n_heads, len_v(=len_k), d_v]
        # print('Q:',Q.size())
        # print('K:',K.size())
        
        #attn_mask = attn_mask.unsqueeze(1).repeat(1, n_heads, 1, 1) 
        # attn_mask : [batch_size, n_heads, seq_len, seq_len]

        # context: [batch_size, n_heads, len_q, d_v], attn: [batch_size, n_heads, len_q, len_k]
        context, attn = ScaledDotProductAttention()(Q, K, V)
        
        context = context.transpose(1, 2).reshape(batch_size, -1, n_heads * d_v) 
    
        # context: [batch_size, len_q, n_heads * d_v]
        output_ = self.fc(context) # [batch_size, len_q, d_model]
       
        #print('attn_output_',output_.size())
        residual.to(device)
        output_.to(device)
        output=self.nor(output_ + residual)
        #output=nn.LayerNorm(d_model)(output_ + residual)
        
        #print('multioutput',(output_ + residual).size())
        #print('2')
        return output, attn
    #返回子层结果和attn

#前向神经网络
class PoswiseFeedForwardNet(nn.Module):
    def __init__(self):
        super(PoswiseFeedForwardNet, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(d_model, d_ff, bias=False),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(d_ff, d_model, bias=False),
            #nn.Dropout()
        )
        self.nor=nn.LayerNorm(d_model)
    def forward(self, inputs):
        '''
        inputs: [batch_size, seq_len, d_model]
        '''
        
        residual = inputs
        #print('input',inputs.size())
        output = self.fc(inputs)
        #print('FF',output.size())
        #print('3')
        return self.nor(output + residual)
    # [batch_size, seq_len, d_model]

class EncoderLayer(nn.Module):
    def __init__(self):
        super(EncoderLayer, self).__init__()
        self.enc_self_attn = MultiHeadAttention()
        self.pos_ffn = PoswiseFeedForwardNet()
        
    def forward(self, Q_inputs,enc_inputs):
        '''
        enc_inputs: [batch_size, src_len, d_model]
        enc_self_attn_mask: [batch_size, src_len, src_len]
        '''
        
        Q_inputs.to(device)
        enc_inputs.to(device)
        # enc_outputs: [batch_size, src_len, d_model], attn: [batch_size, n_heads, src_len, src_len]
        enc_outputs, attn = self.enc_self_attn(Q_inputs, enc_inputs, enc_inputs) 
        #print('multiattn输出:',enc_outputs.size())
        # enc_inputs to same Q,K,V
        #print('enc_outputs',enc_outputs.is_cuda)
        enc_outputs = self.pos_ffn(enc_outputs) 
        #print('ff输出:',enc_outputs.size())
        # enc_outputs: [batch_size, src_len, d_model]
        #print('EncoderLayer的输出：',enc_outputs.size())
        #print('4')
        return enc_outputs, attn
enc_self_attns=[]
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.layers = nn.ModuleList([EncoderLayer() for _ in range(n_layers)])
        self.fc=  nn.Sequential(
            nn.Linear(num_fea*d_model, num_class, bias=False),
            
            nn.Sigmoid()
            #nn.Softmax()
        )
        
    def forward(self,Q_inputs,enc_intputs):
        '''
        enc_inputs: [batch_size, src_len]
        '''
       
        #word_emb = self.src_emb(enc_inputs) # [batch_size, src_len, d_model]
        #pos_emb = self.pos_emb(enc_inputs) # [batch_size, src_len, d_model]
        #enc_outputs = word_emb + pos_emb
        #enc_self_attn_mask = get_attn_pad_mask(enc_inputs, enc_inputs) # [batch_size, src_len, src_len]
        #enc_self_attns = []
        for layer in self.layers:
            # enc_outputs: [batch_size, src_len, d_model], enc_self_attn: [batch_size, n_heads, src_len, src_len]
            enc_intputs, enc_self_attn = layer(Q_inputs,enc_intputs)
            #print('5')
            #print('encoder输出',enc_intputs.size())
            batch_size=enc_intputs.size(0)
            enc_self_attns.append(enc_self_attn)
            #enc_intputs数据嵌入后的维数
        enc_intputs=enc_intputs.view(batch_size,-1)
        #print('分类前的输出',enc_intputs.size())
        outputs=self.fc(enc_intputs)
            
            
            
            #print('outputs',outputs.size())
        #print('6')
        return outputs, enc_self_attns
    
# def preclass(outputs,lamda): #判断标签
#     #print('class_outputs',outputs.size())
#     #print('class_outputs',outputs)
#     max_value, preds=torch.max(outputs,1)
    
#     #print(max_value)
#     value1=max_value-lamda
#     #print(value1)   
#     duibi=value1
#     for i in range(outputs.shape[1]-1):
#         duibi=torch.cat([duibi,value1],0)
#     c=torch.reshape(duibi,(outputs.shape[1],outputs.shape[0]))
#     d=c.transpose(0,1)
#     preds=torch.gt(outputs,d)
#     preds=preds.float()
#     #print('preds',preds)
#     return preds

def preclass(outputs,lamda): #判断标签
    #print(lamda)
    A= pd.read_csv("/kaggle/input/newdata0330/label_infor.csv")
    A=np.array(A) 
    A=A[:,1:]
    a=A
    A=torch.from_numpy(A)
  #找到预测的最大值
    max_value, location=torch.max(outputs,1)
    location=location.cpu().numpy()
    O_=torch.zeros(outputs.shape[0],outputs.shape[1])
    preds=np.zeros((outputs.shape[0],outputs.shape[1]))
    for i in range(0,outputs.shape[0]):
        index=[]
        a1=[]
        #preds[i,location[i]]=1
          #转化概率矩阵
        O_[i,:]=outputs[i,:]/outputs[i,location[i]]-lamda
        #location[i]是第i个样本对应的max索引
        #每个输出概率除以最大输出概率
        preds1=torch.ge(O_[i,:],A[location[i],:])
        preds1=preds1.float()
        #preds1=preds1.numpy()
        preds[i,:]=preds1
        a1=a[location[i],:]
        index=[i for i,x in enumerate(a1) if x==0]
        preds[i,index]=0
        preds[i,location[i]]=1
    #print(preds)
    preds=torch.from_numpy(preds)
    return preds


def newacc(preds,labels):
    #preds=np.cpu().array(preds)
    copy_preds = preds.clone().detach().cpu()
    copy_labels = labels.clone().detach().cpu()
    copy_preds = copy_preds.cpu().numpy()
    copy_labels= copy_labels.cpu().numpy()
    # #labels=np.cpu().array(labels)
    # acc=accuracy_score(copy_labels, copy_preds)
    # acc_num=acc*copy_labels.shape[0]
    # acc_num=int(acc_num)
    acc = np.mean(np.all(np.equal(copy_labels, copy_preds), axis=1).astype("float32"))
    acc_num=acc*copy_labels.shape[0]
    acc_num=int(acc_num)
    #print(acc_num)
    return acc_num

import torch
import torch.nn as nn
import torch.nn.functional as F

def binary_focal_loss(gamma=2, **_):
    def func(input, target):
        assert target.size() == input.size()

        max_val = (-input).clamp(min=0)
        #clamp(min=0),就相当于relu函数的等效，max(0,wx+b)

        loss = input - input * target + max_val + ((-max_val).exp() + (-input - max_val).exp()).log()
        invprobs = F.logsigmoid(-input * (target * 2 - 1))
        #这个
        loss = (invprobs * gamma).exp() * loss
        return loss.mean()

    return func


class FocalLoss(nn.Module):
    # Wraps focal loss around existing loss_fcn(), i.e. criteria = FocalLoss(nn.BCEWithLogitsLoss(), gamma=1.5)
    def __init__(self, loss_fcn, gamma=1.5, alpha=0.25):
        super(FocalLoss, self).__init__()
        self.loss_fcn = nn.BCEWithLogitsLoss()  # must be nn.BCEWithLogitsLoss()
        self.gamma = gamma
        self.alpha = alpha
        self.reduction = loss_fcn.reduction
        self.loss_fcn.reduction = 'none'  # required to apply FL to each element

    def forward(self, pred, true):
        loss = self.loss_fcn(pred, true)
        # p_t = torch.exp(-loss)
        # loss *= self.alpha * (1.000001 - p_t) ** self.gamma  # non-zero power for gradient stability
        
        # TF implementation https://github.com/tensorflow/addons/blob/v0.7.1/tensorflow_addons/losses/focal_loss.py
        pred_prob = torch.sigmoid(pred)  # prob from logits
        p_t = true * pred_prob + (1 - true) * (1 - pred_prob)
        alpha_factor = true * self.alpha + (1 - true) * (1 - self.alpha)
        modulating_factor = (1.0 - p_t) ** self.gamma
        loss *= alpha_factor * modulating_factor

        if self.reduction == 'mean':
            return loss.mean()
        elif self.reduction == 'sum':
            return loss.sum()
        else:  # 'none'
            return loss

def cross_entropy(**_):
    return torch.nn.BCEWithLogitsLoss()


def get_loss(config):
    f = globals().get(config.loss.name)
    return f(**config.loss.params)
# train_pre=[]
# train_output=[]
# train_label=[]
# test_pre=[]
# test_output=[]
# test_label=[]



In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Sun Feb 26 12:15:03 2023

@author: lisha
"""
import numpy as np
from sklearn.metrics import average_precision_score, f1_score, zero_one_loss,hamming_loss,accuracy_score,multilabel_confusion_matrix

epsilon = 1e-8


def compute_f1(precision, recall):
    return 2 * precision * recall / (precision + recall + epsilon)


def example_subset_accuracy(gt, predict):
    ex_equal = np.all(np.equal(gt, predict), axis=1).astype("float32")
    return np.mean(ex_equal)


def example_accuracy(gt, predict):
    ex_and = np.sum(np.logical_and(gt, predict), axis=1).astype("float32")
    ex_or = np.sum(np.logical_or(gt, predict), axis=1).astype("float32")
    return np.mean((ex_and + epsilon) / (ex_or + epsilon))


def example_precision(gt, predict):
    ex_and = np.sum(np.logical_and(gt, predict), axis=1).astype("float32")
    ex_predict = np.sum(predict, axis=1).astype("float32")
    return np.mean((ex_and + epsilon) / (ex_predict + epsilon))


def example_recall(gt, predict):
    ex_and = np.sum(np.logical_and(gt, predict), axis=1).astype("float32")
    ex_gt = np.sum(gt, axis=1).astype("float32")
    return np.mean((ex_and + epsilon) / (ex_gt + epsilon))


def example_f1(gt, predict):
    p = example_precision(gt, predict)
    r = example_recall(gt, predict)
    return (2 * p * r) / (p + r + epsilon)


def _label_quantity(gt, predict):
    tp = np.sum(np.logical_and(gt, predict), axis=0)
    fp = np.sum(np.logical_and(1-gt, predict), axis=0)
    tn = np.sum(np.logical_and(1-gt, 1-predict), axis=0)
    fn = np.sum(np.logical_and(gt, 1-predict), axis=0)
    return np.stack([tp, fp, tn, fn], axis=0).astype("float")


def label_accuracy_macro(gt, predict):
    quantity = _label_quantity(gt, predict)
    tp_tn = np.add(quantity[0], quantity[2])
    tp_fp_tn_fn = np.sum(quantity, axis=0)
    return np.mean((tp_tn + epsilon) / (tp_fp_tn_fn + epsilon))


def label_precision_macro(gt, predict):
    quantity = _label_quantity(gt, predict)
    tp = quantity[0]
    tp_fp = np.add(quantity[0], quantity[1])
    return np.mean((tp + epsilon) / (tp_fp + epsilon))


def label_recall_macro(gt, predict):
    quantity = _label_quantity(gt, predict)
    tp = quantity[0]
    tp_fn = np.add(quantity[0], quantity[3])
    return np.mean((tp + epsilon) / (tp_fn + epsilon))


def label_f1_macro(gt, predict):
    p = label_precision_macro(gt, predict)
    r = label_recall_macro(gt, predict)
    return (2 * p * r) / (p + r + epsilon)


def label_accuracy_micro(gt, predict):
    quantity = _label_quantity(gt, predict)
    sum_tp, sum_fp, sum_tn, sum_fn = np.sum(quantity, axis=1)
    return (sum_tp + sum_tn + epsilon) / (
            sum_tp + sum_fp + sum_tn + sum_fn + epsilon)


def label_precision_micro(gt, predict):
    quantity = _label_quantity(gt, predict)
    sum_tp, sum_fp, sum_tn, sum_fn = np.sum(quantity, axis=1)
    return (sum_tp + epsilon) / (sum_tp + sum_fp + epsilon)


def label_recall_micro(gt, predict):
    quantity = _label_quantity(gt, predict)
    sum_tp, sum_fp, sum_tn, sum_fn = np.sum(quantity, axis=1)
    return (sum_tp + epsilon) / (sum_tp + sum_fn + epsilon)


def label_f1_micro(gt, predict):
    p = label_precision_micro(gt, predict)
    r = label_recall_micro(gt, predict)
    return (2 * p * r) / (p + r + epsilon)


def single_label_accuracy(gt, predict):
    quantity = _label_quantity(gt, predict)
    tp_tn = np.add(quantity[0], quantity[2])
    tp_fp_tn_fn = np.sum(quantity, axis=0)
    return (tp_tn + epsilon) / (tp_fp_tn_fn + epsilon)


def single_label_precision(gt, predict):
    quantity = _label_quantity(gt, predict)
    tp = quantity[0]
    tp_fp = np.add(quantity[0], quantity[1])
    return (tp + epsilon) / (tp_fp + epsilon)


def single_label_recall(gt, predict):
    quantity = _label_quantity(gt, predict)
    tp = quantity[0]
    tp_fn = np.add(quantity[0], quantity[3])
    return (tp + epsilon) / (tp_fn + epsilon)
def write_metrics(gt, predict):
    ex_subset_acc = example_subset_accuracy(gt, predict)
    ex_acc = example_accuracy(gt, predict)
    ex_precision = example_precision(gt, predict)
    ex_recall = example_recall(gt, predict)
    ex_f1 = compute_f1(ex_precision, ex_recall)

    lab_acc_macro = label_accuracy_macro(gt, predict)
    lab_precision_macro = label_precision_macro(gt, predict)
    lab_recall_macro = label_recall_macro(gt, predict)
    lab_f1_macro = compute_f1(lab_precision_macro, lab_recall_macro)

    lab_acc_micro = label_accuracy_micro(gt, predict)
    lab_precision_micro = label_precision_micro(gt, predict)
    lab_recall_micro = label_recall_micro(gt, predict)
    lab_f1_micro = compute_f1(lab_precision_micro, lab_recall_micro)

    score_f1_macro = f1_score(gt, predict, average="macro")
   

    score_f1_micro = f1_score(gt, predict, average="micro")
    

    # hamming loss
    h_loss = hamming_loss(gt, predict)
    
    
    #zero_one_loss
    z_o_loss = zero_one_loss(gt, predict)
    
    
    mAP = average_precision_score(gt, predict)
    
    
    OAA=accuracy_score(gt, predict)
    
    
    Mconfusion_matri=multilabel_confusion_matrix(gt, predict)
    
    mul_matri=Mconfusion_matri.reshape(10,-1)
    return  ex_subset_acc,OAA, ex_acc,ex_precision,ex_recall,ex_f1,lab_acc_macro,lab_precision_macro,lab_recall_macro,lab_f1_macro,lab_acc_micro,lab_precision_micro,lab_recall_micro,lab_f1_micro,score_f1_macro,score_f1_micro,h_loss,z_o_loss,mAP,mul_matri

In [ ]:
import torch
import torch.nn as nn

import torch
import torch.nn as nn
import torch.nn.functional as F


# def FocalLoss(y_pred, y_true, pos_weight, gamma):
#     # y_pred is the logits before Sigmoid
#     assert y_pred.shape == y_true.shape
#     pt = torch.exp(-F.binary_cross_entropy_with_logits(y_pred, y_true, reduction='none')).detach()
#     sample_weight = (1 - pt) ** gamma
#     return F.binary_cross_entropy_with_logits(y_pred, y_true, weight=sample_weight, pos_weight=pos_weight)

class FocalLoss(nn.Module):
    # Wraps focal loss around existing loss_fcn(), i.e. criteria = FocalLoss(nn.BCEWithLogitsLoss(), gamma=1.5)
    def __init__(self, loss_fcn, gamma=1.5, alpha=0.25):
        super(FocalLoss, self).__init__()
        self.loss_fcn = nn.BCEWithLogitsLoss()  # must be nn.BCEWithLogitsLoss()
        self.gamma = gamma
        self.alpha = alpha
        self.reduction = loss_fcn.reduction
        self.loss_fcn.reduction = 'none'  # required to apply FL to each element
        print(f'gamma_alpha',gamma, alpha)
    def forward(self, pred_prob, true):
        loss = self.loss_fcn(pred_prob, true)
        # p_t = torch.exp(-loss)
        # loss *= self.alpha * (1.000001 - p_t) ** self.gamma  # non-zero power for gradient stability
        # TF implementation https://github.com/tensorflow/addons/blob/v0.7.1/tensorflow_addons/losses/focal_loss.py
        #pred_prob = torch.sigmoid(pred)  # prob from logits
        p_t = true * pred_prob + (1 - true) * (1 - pred_prob)
        alpha_factor = true * self.alpha + (1 - true) * (1 - self.alpha)
        modulating_factor = (1.0 - p_t) ** self.gamma
        loss *= alpha_factor * modulating_factor

        if self.reduction == 'mean':
            return loss.mean()
        elif self.reduction == 'sum':
            return loss.sum()
        else:  # 'none'
            return loss

class AsymmetricLoss(nn.Module):
    def __init__(self, gamma_neg=4, gamma_pos=1, clip=0.05, eps=1e-8, disable_torch_grad_focal_loss=True):
        super(AsymmetricLoss, self).__init__()
        self.gamma_neg = gamma_neg
        print(f'gamma_neg',gamma_neg, gamma_pos)
        self.gamma_pos = gamma_pos
        self.clip = clip
        self.disable_torch_grad_focal_loss = disable_torch_grad_focal_loss
        self.eps = eps

    def forward(self, x_sigmoid, y):
        """"
        Parameters
        ----------
        x: input logits
        y: targets (multi-label binarized vector)
        """
        # Calculating Probabilities
        #x_sigmoid = torch.sigmoid(x)
        xs_pos = x_sigmoid
        xs_neg = 1 - x_sigmoid

        # Asymmetric Clipping
        if self.clip is not None and self.clip > 0:
            xs_neg = (xs_neg+self.clip).clamp(max=1)

        # Basic CE calculation
        los_pos = y * torch.log(xs_pos.clamp(min=self.eps))
        los_neg = (1 - y) * torch.log(xs_neg.clamp(min=self.eps))
        loss = los_pos + los_neg

        # Asymmetric Focusing
        if self.gamma_neg > 0 or self.gamma_pos > 0:
            if self.disable_torch_grad_focal_loss:
                torch.set_grad_enabled(False)
            pt0 = xs_pos * y
            pt1 = xs_neg * (1 - y)  # pt = p if t > 0 else 1-p
            pt = pt0 + pt1
            one_sided_gamma = self.gamma_pos * y + self.gamma_neg * (1 - y)
            one_sided_w = torch.pow(1 - pt, one_sided_gamma)
            if self.disable_torch_grad_focal_loss:
                torch.set_grad_enabled(True)
            loss *= one_sided_w

        return -loss.sum()
class Asy1(nn.Module):
    def __init__(self, alpha=0.25,  clip=0.05, eps=1e-8, disable_torch_grad_focal_loss=True):
        super(Asy1,self).__init__()
        self.alpha = alpha
        self.clip = clip
        print(f'alpha_clip',alpha,clip)
        self.disable_torch_grad_focal_loss = disable_torch_grad_focal_loss
        self.eps = eps

    def forward(self, x_sigmoid, y):
        """"
        Parameters
        ----------
        x: input logits
        y: targets (multi-label binarized vector)
        """
        # Calculating Probabilities
        #x_sigmoid = torch.sigmoid(x)
        xs_pos = x_sigmoid
        xs_neg = 1 - x_sigmoid

        # Asymmetric Clipping #添加一个阈值
        if self.clip is not None and self.clip > 0:
            xs_neg = (xs_neg+self.clip).clamp(max=1)

        # Basic CE calculation  基础的BCE计算
        los_pos = y * torch.log(xs_pos.clamp(min=self.eps))
        los_neg = (1 - y) * torch.log(xs_neg.clamp(min=self.eps))
        loss = los_pos + los_neg

        # Asymmetric Focusing
        if self.alpha > 0:
            if self.disable_torch_grad_focal_loss:
                torch.set_grad_enabled(False)
            one_sided_alpha = self.alpha * y + (1-self.alpha) * (1 - y)
            if self.disable_torch_grad_focal_loss:
                torch.set_grad_enabled(True)
            loss *= one_sided_alpha

        return -loss.sum()

class AsymmetricLossOptimized(nn.Module):
    ''' Notice - optimized version, minimizes memory allocation and gpu uploading,
    favors inplace operations'''

    def __init__(self, gamma_neg=4, gamma_pos=1, clip=0.05, eps=1e-8, disable_torch_grad_focal_loss=False):
        super(AsymmetricLossOptimized, self).__init__()

        self.gamma_neg = gamma_neg
        self.gamma_pos = gamma_pos
        self.clip = clip
        self.disable_torch_grad_focal_loss = disable_torch_grad_focal_loss
        self.eps = eps

        # prevent memory allocation and gpu uploading every iteration, and encourages inplace operations
        self.targets = self.anti_targets = self.xs_pos = self.xs_neg = self.asymmetric_w = self.loss = None

    def forward(self, x, y):
        """"
        Parameters
        ----------
        x: input logits
        y: targets (multi-label binarized vector)
        """

        self.targets = y
        self.anti_targets = 1 - y

        # Calculating Probabilities
        #self.xs_pos = torch.sigmoid(x)
        self.xs_pos = x
        self.xs_neg = 1.0 - self.xs_pos

        # Asymmetric Clipping
        if self.clip is not None and self.clip > 0:
            self.xs_neg.add_(self.clip).clamp_(max=1)

        # Basic CE calculation
        self.loss = self.targets * torch.log(self.xs_pos.clamp(min=self.eps))
        self.loss.add_(self.anti_targets * torch.log(self.xs_neg.clamp(min=self.eps)))

        # Asymmetric Focusing
        if self.gamma_neg > 0 or self.gamma_pos > 0:
            if self.disable_torch_grad_focal_loss:
                torch.set_grad_enabled(False)
            self.xs_pos = self.xs_pos * self.targets
            self.xs_neg = self.xs_neg * self.anti_targets
            self.asymmetric_w = torch.pow(1 - self.xs_pos - self.xs_neg,
                                          self.gamma_pos * self.targets + self.gamma_neg * self.anti_targets)
            if self.disable_torch_grad_focal_loss:
                torch.set_grad_enabled(True)
            self.loss *= self.asymmetric_w

        return -self.loss.sum()


class ASLSingleLabel(nn.Module):
    '''
    This loss is intended for single-label classification problems
    '''
    def __init__(self, gamma_pos=0, gamma_neg=4, eps: float = 0.1, reduction='mean'):
        super(ASLSingleLabel, self).__init__()

        self.eps = eps
        self.logsoftmax = nn.LogSoftmax(dim=-1)
        self.targets_classes = []
        self.gamma_pos = gamma_pos
        self.gamma_neg = gamma_neg
        self.reduction = reduction

    def forward(self, inputs, target):
        '''
        "input" dimensions: - (batch_size,number_classes)
        "target" dimensions: - (batch_size)
        '''
        num_classes = inputs.size()[-1]
        log_preds = self.logsoftmax(inputs)
        self.targets_classes = torch.zeros_like(inputs).scatter_(1, target.long().unsqueeze(1), 1)

        # ASL weights
        targets = self.targets_classes
        anti_targets = 1 - targets
        xs_pos = torch.exp(log_preds)
        xs_neg = 1 - xs_pos
        xs_pos = xs_pos * targets
        xs_neg = xs_neg * anti_targets
        asymmetric_w = torch.pow(1 - xs_pos - xs_neg,
                                 self.gamma_pos * targets + self.gamma_neg * anti_targets)
        log_preds = log_preds * asymmetric_w

        if self.eps > 0:  # label smoothing
            self.targets_classes = self.targets_classes.mul(1 - self.eps).add(self.eps / num_classes)

        # loss calculation
        loss = - self.targets_classes.mul(log_preds)

        loss = loss.sum(dim=-1)
        if self.reduction == 'mean':
            loss = loss.mean()

        return loss


In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Thu Feb 23 16:08:30 2023

@author: lisha
"""
import torch
import torch.nn as nn
import numpy as np
import pandas as pd




#3个融合
class my_Data_Set3(nn.Module):
    
    def __init__(self, num_fea,m,infor1,infor2,infor3,lbp):
        super(my_Data_Set3, self).__init__()
        # 打开存储图像名与标签的txt文件
        self.images = infor1[:,1] #图像名
        self.feature1=infor1[:,2:m-5]
        self.feature2=infor2[:,2:m-5]
        self.feature3=infor3[:,2:m-5]
        self.lbp=lbp
        self.labels = infor1[:,m-5:]
        self.num_fea=num_fea
        print('3个融合')
    def __getitem__(self, item):
        # 获取图像名和标签
        imageName = self.images[item]
        label = self.labels[item]
       # 读入图像信息
        feature1=self.feature1[item]
        feature2=self.feature2[item]
        feature3=self.feature3[item]
        lbp=self.lbp[item]
        feature1= pd.to_numeric(feature1) #object格式转为数值格式
        feature2= pd.to_numeric(feature2)
        feature3= pd.to_numeric(feature3)
        label = pd.to_numeric(label)
        lbp= pd.to_numeric(lbp)
        feature_=np.append(feature1,feature2)
        feature_=np.append(feature_,feature3)
        feature_=np.matrix(feature_)
        lbp=np.matrix(lbp)
       #格式的转换
        lbp=lbp.reshape(1,-1)
        feature=feature_.reshape(self.num_fea,-1)
       # 需要将标签转换为float类型，BCELoss只接受float类型
        label = torch.FloatTensor(label)
        feature= torch.FloatTensor(feature)
        lbp= torch.FloatTensor(lbp)
       # print(lbp.shape)
       # print(feature.shape)
       # print(label.shape)
        return feature,lbp,label
 
    # 重写这个函数，来看数据集中含有多少数据
    def __len__(self):
        return len(self.images)



In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#数据集整理
alexnet128_train= pd.read_csv("/kaggle/input/newdata0330/new_alexnet128_train_all.csv")
alexnet128_train=np.array(alexnet128_train) 
alexnet128_test= pd.read_csv("/kaggle/input/newdata0330/new_alexnet128_test_all.csv")
alexnet128_test=np.array(alexnet128_test) 

densent128_train= pd.read_csv("/kaggle/input/newdata0330/new_densnet128_train_all.csv")
densent128_train=np.array(densent128_train) 
densent128_test= pd.read_csv("/kaggle/input/newdata0330/new_densnet128_test_all.csv")
densent128_test=np.array(densent128_test) 

vgg128_train= pd.read_csv("/kaggle/input/newdata0330/new_vgg128_train_all.csv")
vgg128_train=np.array(vgg128_train) 
vgg128_test= pd.read_csv("/kaggle/input/newdata0330/new_vgg128_test_all.csv")
vgg128_test=np.array(vgg128_test) 

resnet128_train= pd.read_csv("/kaggle/input/newdata0330/new_resnet128_train_all.csv")
resnet128_train=np.array(resnet128_train) 
resnet128_test= pd.read_csv("/kaggle/input/newdata0330/new_resnet128_test_all.csv")
resnet128_test=np.array(resnet128_test)


lbp_train= pd.read_csv("/kaggle/input/newdata0330/new_lbp_train.CSV")
#lbp_train=pd.DataFrame(lbp_train,dtype=np.float)；
lbp_test= pd.read_csv("/kaggle/input/newdata0330/new_lbp_test.CSV")
lbp_train=np.array(lbp_train)
lbp_test=np.array(lbp_test)

m=resnet128_train.shape[1]
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
num_class=5
# Transformer Parameters
d_model =128 # Embedding Size,512-1024,128-512
q_model=lbp_test.shape[1]
d_ff = 1024# FeedForward dimension
n_layers = 4  # number of Encoder of Decoder Layer
n_heads = 4  # number of heads in Multi-Head Attention
d_k = d_v =int(d_model/n_heads) # dimension of K(=Q), V
batch_size=50
#一些超参数的设计
a=4   #控制不对称的顺势函数，gamma_neg
b=1  #控制不对称的顺势函数，gamma_pos


# 定义自己数据集的数据读入类
#order=[resnet,densent,vgg,alexnet]
order1=1
order2=1
order3=1
order4=0
num_fea=3
train_Data = my_Data_Set3(num_fea,m,resnet128_train,densent128_train,alexnet128_train,lbp_train)
test_Data =my_Data_Set3(num_fea,m,resnet128_test,densent128_test,alexnet128_test,lbp_test)

#train_Data = my_Data_Set(vgg128_train,alexnet128_train,lbp_train)
# 读取数据集大小
dataset_sizes = {'train': train_Data.__len__(), 'test': test_Data.__len__()}
train_DataLoader = DataLoader(train_Data, batch_size, shuffle=True)
test_DataLoader = DataLoader(test_Data, batch_size, shuffle=True)
dataloaders = {'train':train_DataLoader, 'test':test_DataLoader}

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
def train_model(model, parameters, dtype, device,num_epochs):
    torch.cuda.manual_seed_all(1)
    since = time.time()
    #active_fun = nn.Sigmoid()
    #best_model_wts = copy.deepcopy(model.state_dict())
    best_model_wts=[]
    best_acc = 0.0
    train_acc=0.0
    num_epoch=0
    train_pre_best=[]
    train_output_best=[]
    train_label_best=[]
    test_pre_best=[]
    test_output_best=[]
    test_label_best=[]
    #optimizer = optim.SGD(net.parameters(), # or any optimizer you prefer 
  #                       lr=parameters.get("lr", 0.001), # 0.001 is used if no lr is specified
  #                       momentum=parameters.get("momentum", 0.9)
  # )
    alpha=parameters.get("alpha", 0.05)/20
    clip=parameters.get("clip", 0.01)/100
    lamda=parameters.get("lamda", 0.15)/100
    criterion = Asy1(alpha,clip).to(device)
    #criterion = Asy1(alpha=aa, clip=c)
    optimizer= optim.SGD(model.parameters(), lr=0.001, momentum=0.4)
    #optimizer=optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)
    scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
    for epoch in range(num_epochs):
        train_pre=[]
        train_output=[]
        train_label=[]
        test_pre=[]
        test_output=[]
        test_label=[]
        #ytesttmp=np.ones((1,num_class))*0.5
        train_pre=np.ones((1,num_class))*0.5
        train_output=np.ones((1,num_class))*0.5
        train_label=np.ones((1,num_class))*0.5
        test_pre=np.ones((1,num_class))*0.5
        test_output=np.ones((1,num_class))*0.5
        test_label=np.ones((1,num_class))*0.5
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)
        # Each epoch has a training and validation phase
        for phase in ['train', 'test']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode#模型评估
            running_loss = 0.0
            running_corrects = 0
            # Iterate over data.
            for deepinputs,trainputs,labels in dataloaders[phase]:
                deepinputs = deepinputs.to(device)
                trainputs = trainputs.to(device)
                labels = labels.to(device)
                #print('deepinputs',deepinputs.is_cuda)
                #print('输入trainputs：',trainputs.size())
                #print('输入deepinputs：',deepinputs.size())
                #print('输入labels：',labels.size())
                # zero the parameter gradients
                optimizer.zero_grad()
                # forward
                # track history if only in train
                #model.to(device)
                #print('model',model.is_cuda)
                with torch.set_grad_enabled(phase == 'train'):
                    trainputs.to(device)
                    deepinputs.to(device)
                    #print('trainputs',trainputs.is_cuda)
                    model.to(device)
                    outputs,enc_self_attns= model(trainputs,deepinputs)
                    #这一条命令有问题
                    #print('outputs',outputs.is_cuda)
                    #print('输出outputs：',outputs.size())
                    #print(outputs.size())
                    outputs = outputs.to(device)
                    loss = criterion(outputs, labels)
                    # batch_size1=inputs.size(0)
                    # outputs=outputs.view(batch_size1,num_class)
                    preds=preclass(outputs,lamda)
                    #loss = criterion(outputs, labels)
                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                    
                # statistics
                running_loss += loss.item() * deepinputs.size(0)
                #running_corrects += torch.sum(preds == labels.data)
                running_corrects += newacc(preds,labels.data)
                #print('running_corrects',running_corrects)
                if phase == 'train':
                    #print('preds1',preds.is_cuda)
                    preds=preds.detach().cpu().numpy()
                    #print('preds2',preds.is_cuda)
                    outputs=outputs.detach().cpu().numpy()
                    labels=labels.detach().cpu().numpy()
                    train_pre=np.vstack((train_pre,preds))
                    train_output=np.vstack((train_output,outputs))
                    train_label=np.vstack((train_label,labels))
                    #train_pre=train_pre.to(device)

                if phase == 'test':
                    preds=preds.detach().cpu().numpy()
                    outputs=outputs.detach().cpu().numpy()
                    labels=labels.detach().cpu().numpy()
                    test_pre=np.vstack((test_pre,preds))
                    test_output=np.vstack((test_output,outputs))
                    test_label=np.vstack((test_label,labels))
            if phase == 'train':
                scheduler.step()
            epoch_loss = running_loss / dataset_sizes[phase]
            #torch.sum(preds == labels.data)
            #epoch_acc = running_corrects.double() / dataset_sizes[phase]
            if phase == 'train':
                epoch_train_acc = running_corrects / dataset_sizes[phase]
                epoch_acc=epoch_train_acc
            if phase == 'test':
                epoch_test_acc = running_corrects / dataset_sizes[phase]
                epoch_acc=epoch_test_acc
            print('running_corrects',running_corrects)
            print(f'dataset_sizes{phase}',dataset_sizes[phase])
            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
            # deep copy the model,val或者train
            
            if phase == 'test' and epoch_test_acc>= best_acc:
                best_acc = epoch_test_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                best_epoch=num_epoch
                train_acc = epoch_train_acc
                train_pre_best=train_pre
                train_output_best=train_output
                train_label_best=train_label
                test_pre_best=test_pre
                test_output_best=test_output
                test_label_best=test_label
            
                
        print()
        num_epoch=num_epoch+1
    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'train Acc: {train_acc:4f}')
    print(f'Best test Acc: {best_acc:4f}')
    print(f'best_epoch: {best_epoch:4f}')
    
    # load best model weights
    model.load_state_dict(best_model_wts)
    
    return alpha,clip,lamda,model,train_pre_best,train_output_best,train_label_best,test_pre_best,test_output_best,test_label_best


In [ ]:
a_result_ASL_=[]

def init_net(parameterization):
    cudnn.benchmark = False            # if benchmark=True, deterministic will be False
    cudnn.deterministic = True
    seed=1
    torch.manual_seed(seed)            # 为CPU设置随机种子
    torch.cuda.manual_seed(seed)       # 为当前GPU设置随机种子
    torch.cuda.manual_seed_all(seed)   # 为所有GPU设置随机种子
    random.seed(seed)
    np.random.seed(seed)
    model=Encoder() #
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model=model.to(device)

    # The depth of unfreezing is also a hyperparamete
    return model # return untrained model
#最后，我们需要一个train_evaluate()函数，该函数在每次运行时都会被贝叶斯优化器调用。
# 优化器在parameterization化中生成一组新的超parameterization ，将其传递给此函数，
#然后分析返回的评估结果。
a_result_ASL_=[]
def train_evaluate(parameterization):
    # constructing a new training data loader allows us to tune the batch size
    # Get neural net
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    untrained_net = init_net(parameterization) 
    untrained_net.to(device)
    # train
    #trained_net = train_model(net=untrained_net, train_loader=train_loader, 
                            # parameters=parameterization, dtype=dtype, device=device)
    alpha,clip,lamda,trained_net,train_pre,train_output,train_label,test_pre,test_output,test_label = train_model(untrained_net, parameters=parameterization, dtype=dtype, device=device,num_epochs=40)
    
    # return the accuracy of the model as it was trained in this run
    #return evaluate(
    #     net=trained_net,
    #     data_loader=testloader,
    #     dtype=dtype,
    #     device=device,
    # )
    ex_subset_acc,OAA,ex_acc,ex_precision,ex_recall,ex_f1,lab_acc_macro,lab_precision_macro,lab_recall_macro,lab_f1_macro,lab_acc_micro,lab_precision_micro,lab_recall_micro,lab_f1_micro,score_f1_macro,score_f1_micro,h_loss,z_o_loss,mAP,mul_matri=write_metrics(test_label[1:,:],test_pre[1:,:])
    a_result_ASL=[alpha,clip,lamda,ex_subset_acc, OAA,ex_acc,ex_recall,ex_f1,lab_acc_macro,lab_precision_macro,lab_recall_macro,lab_f1_macro,lab_acc_micro,lab_precision_micro,lab_recall_micro,lab_f1_micro,score_f1_macro,score_f1_micro,h_loss,z_o_loss,mAP]
    a_result_ASL_.append(a_result_ASL)
    a_result_ASL_1=pd.DataFrame(a_result_ASL_)
    a_result_ASL_1.to_csv('/kaggle/working/result_SGD128_3_2.csv')
    #ex_subset_acc.to(device)
    #print(ex_subset_acc)
    return ex_subset_acc
# model,train_pre_best,train_output_best,train_label_best,test_pre_best,test_output_best,test_label_best

#dtype = torch.float
#优化操作
#torch.cuda.set_device(0) #this is sometimes necessary for me
dtype = torch.int
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
 
best_parameters, values, experiment, model = optimize(
    parameters=[
        {"name": "alpha", "type": "range", "bounds": [4, 14],},
        {"name": "clip", "type": "range", "bounds": [1, 10]},
        {"name": "lamda", "type": "range", "bounds": [1, 20]},
        #{"name": "max_epoch", "type": "range", "bounds": [1, 30]},
        #{"name": "stepsize", "type": "range", "bounds": [20, 40]},        
    ],
  
    evaluation_function=train_evaluate,
    objective_name='Subset_Accuracy',
)
#print(parameters.is_cuda)
print(best_parameters)
means, covariances = values
print(means)
print(covariances)

In [ ]:
best_objectives = np.array([[trial.objective_mean*100 for trial in experiment.trials.values()]])
 
best_objective_plot = optimization_trace_single_method(
    y=np.maximum.accumulate(best_objectives, axis=1),
    #title="Model performance vs. # of iterations",
    ylabel="Subset_Accuracy %",
)
render(best_objective_plot)
#plt.savefig("best_objective_plot.jpg")
render(plot_contour(model=model, param_x='alpha', param_y='clip', metric_name='Subset_Accuracy'))
#plt.savefig("performance.jpg")

In [ ]:
plt.savefig("best_objective_plot.jpg")
render(plot_contour(model=model, param_x='alpha', param_y='clip', metric_name='Subset_Accuracy'))